In [0]:
import pandas as pd
import numpy as np
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import avg, current_date, col, year, date_diff,floor, count
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import min, max, sum, lit, desc, asc
from pyspark.sql.functions import col, upper, substring, when
     


In [0]:
df_pitstops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv',header=True)
df_pitstops.display()

###Q1:[10 pts] What was the average time each driver spent at the pit stop for each race?


In [0]:
# Cast columns to FloatType
df_pitstops = df_pitstops.withColumn('milliseconds', df_pitstops['milliseconds'].cast(FloatType()))
display(df_pitstops)

In [0]:

avg_pitstop_time = df_pitstops.groupBy('driverId','raceId').agg(avg(col('milliseconds')).alias('avg_pitstop_time'))
avg_pitstop_time.display()

###Q2:[20 pts] Rank the average time spent at the pit stop in order of who won each race

In [0]:
race_results = spark.read.csv('s3://columbia-gr5069-main/raw/results.csv', header=True)
display(race_results)

In [0]:
#want to combine race_results with avg_pitstop_time
pitstops_driver_avg_time = avg_pitstop_time.join(race_results, on=['raceId','driverId'], how='inner')
display(pitstops_driver_avg_time)

In [0]:
#want to see the average pit stop time for each driver for each race depending on his position
rank_avg=pitstops_driver_avg_time.groupBy('raceId','driverId','rank').agg(avg(col('avg_pitstop_time')).alias('avg_pit_milliseconds'))
#want ascending order

rank_avg=rank_avg.orderBy('rank', ascending=True)
display(rank_avg)

###3.[20 pts] Insert the missing code (e.g: ALO for Alonso) for drivers based on the 'drivers' dataset


In [0]:
df_drivers= spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', 
                            header=True)
display(df_drivers)

In [0]:
%python
# Update the 'code' column
df_drivers = df_drivers.withColumn('code', upper(substring(col('surname'), 1, 3)))

# Create the 'code_complete' column
df_drivers = df_drivers.withColumn('code_complete', when(col('code') == '/N', 'N').otherwise('Y'))

# Display the result
display(df_drivers)

###4.[20 pts] Who is the youngest and oldest driver for each race? Create a new column called “Age”


In [0]:
#First i will calculate the age using drivers dataframe and the column dob
from pyspark.sql.functions import current_date, datediff, col

df_drivers = df_drivers.withColumn('age', (datediff(current_date(), col('dob')) / 365.25).cast('int'))

# Display the result
display(df_drivers)

#I could have divided by 365 but the 0.25 takes leap years into account

In [0]:
#now i want to join the race results with the drivers dataframe
# Join the race_results with the df_drivers dataframe
df_joined = race_results.join(df_drivers, on='driverId', how='inner')

# Display the result
display(df_joined)

In [0]:
#After having joined the two dataframes, i will calculate the youngest and oldest driver for each race one by one
#for youngest
# Calculate the youngest and oldest driver for each race
df_age_youngest = df_joined.groupBy('raceId', 'surname').agg(
    min('age').alias('youngest_age'),
)

# Display the result
display(df_age_youngest)



In [0]:
# Calculate the youngest and oldest driver for each race
df_age_oldest = df_joined.groupBy('raceId', 'surname').agg(
    max('age').alias('oldest_age'),
)

# Display the result
display(df_age_oldest)

In [0]:
%python

# Calculate the youngest and oldest driver for each race and surname
df_age_extremes = df_joined.groupBy('raceId', 'surname').agg(
    min('age').alias('youngest_age'),
    max('age').alias('oldest_age')
)

# Display the result
display(df_age_extremes)

###5.[20 pts] For a given race, which driver has the most wins and losses?

In [0]:

#loading driver_standings data
df_wins= spark.read.csv('s3://columbia-gr5069-main/raw/driver_standings.csv', 
                            header=True)
display(df_wins)

In [0]:
# Create a new column 'loss' where all non-1 values in 'wins' are considered as a loss
df_wins = df_wins.withColumn('loss', when(col('wins') != 1, 1).otherwise(0))

# Group by 'raceId' and 'driverId' and sum the number of wins for each driver in each race
df_win_count = df_wins.groupBy('raceId', 'driverId').agg(sum(col('wins')).alias('wins_count'))

df_loss_count = df_wins.groupBy('raceId', 'driverId').agg(sum(col('loss')).alias('losses_count'))

# Display the result
display(df_win_count)
display(df_loss_count)


In [0]:
# Join the wins and losses DataFrames
df_win_loss = df_win_count.join(df_loss_count, on=['raceId', 'driverId'], how='outer')
display(df_win_loss)

In [0]:
#want surnames to know which driver ID belongs to which driver so should join the two dataframes df_drivers, df_wins_losses using raceID and driverID
df_standings = df_win_loss.join(df_drivers, on=['driverId'], how='outer')
display(df_standings)




In [0]:
#group wins and losses by race id driver id and surame
df_standings = df_standings.groupBy('raceId', 'driverId', 'surname').agg(sum('wins_count').alias('wins_count'), sum('losses_count').alias('losses_count'))
display(df_standings)

###6.[10 pts] Continue exploring the data by answering your own question.
My Question:Which drivers have won and lost the most races overall?

In [0]:
df_sum_win=df_standings.groupBy('surname', 'driverId').agg(sum('wins_count').alias('wins_count'))
#want descending order
df_sum_win=df_sum_win.orderBy(col('wins_count').desc())
display(df_sum_win)

In [0]:
df_sum_loss=df_standings.groupBy('surname', 'driverId').agg(sum('losses_count').alias('losses_count'))
#want descending order
df_sum_loss=df_sum_loss.orderBy(col('losses_count').desc())
display(df_sum_loss)